# Approve and Deploy the Model

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

![Pipeline](img/generative_ai_pipeline_rlhf_plus.png)

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=32877658112, available=5238755328, percent=84.1, used=27166175232, free=561221632, active=30697627648, inactive=377290752, buffers=0, cached=5150261248, shared=1798144, slab=494522368)


In [3]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

In [4]:
%store -r role

# List Pipeline Execution Steps


In [5]:
%store -r pipeline_name

In [6]:
print(pipeline_name)

dialogue-summary-pipeline-1682447409


In [7]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:079002598131:pipeline/dialogue-summary-pipeline-1682447409/execution/5scb5knzi0d3',
  'PipelineExecutionDisplayName': 'execution-1682447418728',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2023, 4, 25, 18, 30, 18, 624000, tzinfo=tzlocal())}]
CPU times: user 12.7 ms, sys: 0 ns, total: 12.7 ms
Wall time: 288 ms


In [8]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [9]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:079002598131:pipeline/dialogue-summary-pipeline-1682447409/execution/5scb5knzi0d3


In [10]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'AttemptCount': 0,
                             'EndTime': datetime.datetime(2023, 4, 25, 18, 53, 41, 237000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:079002598131:model-package/summarization-1682447415/1'}},
                             'StartTime': datetime.datetime(2023, 4, 25, 18, 53, 40, 301000, tzinfo=tzlocal()),
                             'StepName': 'Summarization-RegisterModel',
                             'StepStatus': 'Succeeded'},
                            {'AttemptCount': 0,
                             'EndTime': datetime.datetime(2023, 4, 25, 18, 53, 41, 628000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:079002598131:model/pipelines-5scb5knzi0d3-createmodel-nlidmy7zb5'}},
                             'StartTime': datetime.datetime(2023, 4, 25, 18, 53, 40, 301000, tzinfo=tzlocal()),
                    

# View Registered Model

In [11]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "Summarization-RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

arn:aws:sagemaker:us-east-1:079002598131:model-package/summarization-1682447415/1


# Approve the Model for Deployment

In [12]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",  # Other options are Rejected and PendingManualApproval
)

# View Created Model

In [13]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

pipeline_model_name = model_arn.split("/")[-1]
print(pipeline_model_name)

arn:aws:sagemaker:us-east-1:079002598131:model/pipelines-5scb5knzi0d3-createmodel-nlidmy7zb5
pipelines-5scb5knzi0d3-createmodel-nlidmy7zb5


# Create Model Endpoint from Model Registry
More details here:  https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html


In [14]:
import time

timestamp = int(time.time())

model_from_registry_name = "model-from-registry-{}".format(timestamp)
print("Model from registry name : {}".format(model_from_registry_name))

model_registry_package_container = {
    "ModelPackageName": model_package_arn,
}

Model from registry name : model-from-registry-1682449391


In [15]:
from pprint import pprint

create_model_from_registry_response = sm.create_model(
    ModelName=model_from_registry_name, ExecutionRoleArn=role, PrimaryContainer=model_registry_package_container
)
pprint(create_model_from_registry_response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:079002598131:model/model-from-registry-1682449391',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '92',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Tue, 25 Apr 2023 19:03:12 GMT',
                                      'x-amzn-requestid': '03ad288b-5f6e-4586-82ea-cc7280397101'},
                      'HTTPStatusCode': 200,
                      'RequestId': '03ad288b-5f6e-4586-82ea-cc7280397101',
                      'RetryAttempts': 0}}


In [16]:
model_from_registry_arn = create_model_from_registry_response["ModelArn"]
model_from_registry_arn

'arn:aws:sagemaker:us-east-1:079002598131:model/model-from-registry-1682449391'

#### Note: In this workshop, we are intentionally deploying our model to only 1 instance. The general recommendation is to deploy to 2 or more instances for automatic placement across two AZs for high availability.

In [17]:
endpoint_config_name = "model-from-registry-epc-{}".format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g5.12xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": pipeline_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

model-from-registry-epc-1682449391


In [18]:
%store -d pipeline_endpoint_name

In [19]:
# Pick up the existing `pipeline_endpoint_name` if it was already created
%store -r pipeline_endpoint_name

no stored variable or alias pipeline_endpoint_name


In [20]:
timestamp = int(time.time())
pipeline_endpoint_name = "model-from-registry-ep-{}".format(timestamp)
print("Created Pipeline EndpointName={}".format(pipeline_endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

Created Pipeline EndpointName=model-from-registry-ep-1682449393
arn:aws:sagemaker:us-east-1:079002598131:endpoint/model-from-registry-ep-1682449393


In [21]:
%store pipeline_endpoint_name

Stored 'pipeline_endpoint_name' (str)


In [22]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker HTTPS Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_
_Note:  This will take a few minutes.  Please be patient._

In [ ]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)